<h1>Machine Learning Internship | Navia Life Care</h1>

Name: Uma T V

Email: uma.tv1699@gmail.com

Roll No: ME17B170

Institute: Indian Institute of Technology Madras

<h2>Part 3 out of 3</h2>

<h3> 03 Predicting Test Tagnames </h3>
<p style="color:blue"> This program uses the most efficient model from program 2 and uses that to predict the tagnames of the test images given in electoral-captchas, through individual number splitting analysis and stores the predicted values as filenames of the same images in another folder Predicted_Captchas </p>

In [1]:
# Importing Libraries

import numpy as np
import pandas as pd
import sklearn 
import statsmodels.api as sm
from sklearn.cluster import k_means
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
import os
from matplotlib.pyplot import imread
from PIL import Image
import cv2

In [2]:
# Reading the split and processed individual numbers pixel information

train=pd.read_csv('split_train.csv')

In [3]:
# Seprating into pixel values (features) and labels

y_train=train['120']
x_train=train.drop('120',axis=1)

In [4]:
# Defining a Pipeline with all the models

pipe = Pipeline([('pca',IncrementalPCA(n_components=10)),
                 ('svc', GridSearchCV(estimator = SVC(), param_grid = {"C": [5]}, scoring= 'accuracy', 
                                      cv = KFold(n_splits=3,shuffle=True,random_state=42),
                                      verbose = 1, return_train_score=True))])

In [5]:
# Fitting the model to the data

pipe.fit(x_train, y_train)  

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s finished


Pipeline(memory=None,
         steps=[('pca',
                 IncrementalPCA(batch_size=None, copy=True, n_components=10,
                                whiten=False)),
                ('svc',
                 GridSearchCV(cv=KFold(n_splits=3, random_state=42, shuffle=True),
                              error_score=nan,
                              estimator=SVC(C=1.0, break_ties=False,
                                            cache_size=200, class_weight=None,
                                            coef0=0.0,
                                            decision_function_shape='ovr',
                                            degree=3, gamma='scale',
                                            kernel='rbf', max_iter=-1,
                                            probability=False,
                                            random_state=None, shrinking=True,
                                            tol=0.001, verbose=False),
                              iid='deprecate

In [6]:
# Saving the model pipeline

pickle.dump(pipe, open("Model.sav", 'wb'))

In [7]:
# a function to convert rgb to grayscale

def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [8]:
# a function to load images along with their pixel informations to lists

def load_images_from_folder(folder):
    labels=[]
    images = []
    untouched_images=[]
    for filename in os.listdir(folder):
        untouched_image=cv2.imread(os.path.join(folder,filename))
        img = rgb2gray(imread(os.path.join(folder,filename)))
        untouched_images.append(untouched_image)
        images.append(img)     
    return images, untouched_images

In [9]:
# loading the test images (untagged to be predicted ones)

test_images, test_untouched_images =load_images_from_folder('electoral-captchas')

In [10]:
# creating a new directory to save the predicted images with their correct tag as their filename

if not os.path.exists('Predicted_Captchas'):
    os.makedirs('Predicted_Captchas')


In [11]:
#specifying the number of rows and columns in split number mini images 

rows=12
cols=10

In [12]:
# predicting the tag of test images by splitting them into indivual numbers 
#storing the images along with their correc filename in Predicted_Captchas

for i in range(len(test_images)):
    split_test= pd.DataFrame(columns=[i for i in range(rows*cols)])
    df=pd.DataFrame()
    arr_prev=0
    arr=0
    break_=[]
    for j in range(test_images[i].shape[1]):
        df[j]=test_images[i][:,j]
        values=df[j].unique()
        sig=0
        for k in values:
            if(k<0.5): sig=1
        if(sig==0): arr=0
        else: arr=1
        if(arr==1 and arr_prev==0):
            if( len(break_)==0 or (j-1) > (break_[-1]+cols)): break_.append(j-1)
        arr_prev=arr
    df_list=[]
    if(len(break_)!=5): 
        print("ERRORRR")
        print(i)
    for j in range(len(break_)):
        if(break_[j]+cols>130): break_[j]=129-cols
        df_list.append(df[df.columns[break_[j]:break_[j]+cols]])
        break_row=0
        for k in range(42):
            for l in df_list[j].columns:
                if(df_list[j][l][k]<0.5):
                    break_row=k
        df_list[j]=df_list[j].loc[break_row-10:break_row+1]
    for j in range(len(df_list)):
        pixels=[]
        for k in df_list[j].columns:
            for l in df_list[j].index:
                pixels.append(df_list[j][k][l])
        temp_df=pd.DataFrame([pixels],columns=[i for i in range(rows*cols)])
        split_test=pd.concat([split_test,temp_df])
    if(i%100==0): print(i,"done")
    test_predict = pipe.predict(split_test)
    filename=''
    for j in range(len(test_predict)):
        filename=filename+str(test_predict[j])
    cv2.imwrite('Predicted_Captchas/'+filename+'.png',test_untouched_images[i])

0 done
100 done
200 done
300 done
400 done
500 done
600 done
700 done
800 done
900 done
1000 done
1100 done
1200 done
1300 done
1400 done


<p style="color:blue">Hence, we have successfully predicted the tagnames of the captchas given in electoral_captchas and we have stored the same images with their correct filenames in a new folder Predicted_Captchas. The predictions , on manual inspection are almost 100 percent accurate</p>